In [4]:
import urllib as urllib
import gradio as gr
import json
import openai
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import configparser
import time
from astral import sun,Observer
import pickle
import datetime as dt
import pandas as pd

from datasets import load_dataset




In [2]:
config = configparser.ConfigParser()
config.read('config.ini')


openai.api_key =config['DEFAULT']['openai_api_key']
#weather_api_key = config['DEFAULT']['weather_api_key']

memcache=None

startUpTime = dt.datetime.now()

In [3]:

def clearMemcache(memcache):
    if len(memcache)==0:
        return memcache
    for k in memcache.keys():
        if ((dt.datetime.now()-memcache[thisKey]['timestamp']).seconds)/60>10:
            del(memcache[k])
    return memcache


def checkMemcache(memcache,thisKey):
    if thisKey in memcache:
        if ((dt.datetime.now()-memcache[thisKey]['timestamp']).seconds)/60<10:
            return memcache[thisKey]['content']
    return None

def loadMemcache(memcache,thisKey,content):
    memcache[thisKey]={}
    memcache[thisKey]['timestamp']=dt.datetime.now()
    memcache[thisKey]['content']=content
    saveMemcache(memcache)
    return memcache

def saveMemcache(memcache):
    file = open('memcache', 'wb')
    memcache = pickle.dump(memcache, file)
    file.close()

if memcache is None:
    try:
        file = open('memcache', 'rb')
        memcache = pickle.load(file)
        file.close()
        clearMemcache(memcache)
    except:
        memcache={}






In [8]:

dataset = load_dataset("scikit-learn/auto-mpg")
df=pd.DataFrame(dataset['train'])


Generating train split: 0 examples [00:00, ? examples/s]

-2

In [297]:


def aggGroupBy(target,groupList=None,functionList=['mean'],filterConditions=None,precision=2):
    if type(groupList)!=list:
        groupList = [groupList]
    if type(functionList)!=list:
        functionList=[functionList]
    if groupList is None:
        report=df.agg({target:functionList}).round(precision)
    if filterConditions is None:
        report=df.groupby(groupList).agg({target:functionList}).round(precision)
    else:
        print(filterConditions)
        report=df.groupby(groupList).agg({target:functionList}).round(precision)
    return report.head(20)


def minOrMaxCase(target,aggFunction):
    if aggFunction == 'max':
        rowOrder = False
    else:
        rowOrder=True
    if target=='acceleration' and rowOrder==False:
        rowOrder = True
    elif target=='acceleration':
        rowOrder = False
    report=df.sort_values(target,ascending=rowOrder).head(1)
    report = json.dumps(report.to_dict(orient='records')[0])
    return report


functions = [
    {
        "name": "aggGroupBy",
        "description": '''Makes an aggregated report from the dataset.
          Lets the user specify a variable to be analyzed, one or more grouping variables, 
          and one or more functions.  You can also use pandas to filter the dataframe
          by specifying filterConditions.''',
        "parameters": {
            "type": "object",
            "properties": {
                "groupList": {
                    "type": "string",
                    "description": "List of variable names to group by",
                },
                "target": {"type": "string","description":"the variable name to be analyzed and aggregated"},
            },
            "functionList": {"type": "string","description":"list of pandas aggregation functions"},
            
            "filterConditions": {"type": "string","description":"pandas where conditions to filter the dataset with"},
            },
            "required": ["target"],
        },
    {
        "name": "minOrMaxCase",
        "description": '''Finds one case from the dataframe with a max or min value on the specified target variable.''',
        "parameters": {
            "type": "object",
            "properties": {
                "aggFunction": {
                    "type": "string",
                    "description": "Specifies if the user wants the argmax or argmin.  return either 'min' or 'max'",
                },
                "target": {"type": "string","description":"the variable name to be analyzed and aggregated"},
            },
            "functionList": {"type": "string","description":"list of pandas aggregation functions"},
            
            "filterConditions": {"type": "string","description":"pandas where conditions to filter the dataset with"},
            },
            "required": ["target"],
        }
    
]



In [309]:
varnames = ','.join(list(df.columns))


messages = [
        {  
        "role": "system",
        "content": """You are a helpful assistant that analyzes a dataset for the user. You can use
        function calls to get data, or respond in sentences when proovided the data.
        If you don't know how to apply to a function, apoligize and say you don't know how to do that.
          You can answer questions about these variable names :"""+varnames+"""
        All parameter values must be one of the specified variable names.
        Do not make up information, and keep your responses concise.
        If you identify the need to reply with a function call, but some required parameter are missing,
          reply with a follow up question asking for missing parameters"""
    },
    {'role':"user",
     'content':"Make a report of mileage by number of cylinders. Give me average, min and max mileage per number of cylinders."}
]


In [310]:
#messages.append({'role':'user','content':'what is the average mileage by year?'})
response = callLLM(messages,functions)
cr = classifyReponse(response)
cr

'function_call'

In [311]:
#run a function to check if its got the right info, and send back if the function call doesnt make sense.  

In [312]:
f = response['choices'][0]['message']['function_call']['name']
kwargs = response['choices'][0]['message']['function_call']['arguments']
kwargs = json.loads(kwargs)
print(kwargs,f)
function_response = globals()[f](**kwargs)

{'groupList': 'cylinders', 'target': 'mpg'} aggGroupBy


In [314]:
print(kwargs)
print(function_response)

{'groupList': 'cylinders', 'target': 'mpg'}
             mpg
            mean
cylinders       
3          20.55
4          29.29
5          27.37
6          19.99
8          14.96


In [308]:
messages.append({'role':'system','content':'''use the following data to concisely answer the user's question above accurately.
                 Provide one or two other interesting facts from the JSON data.
                 Do not use a function call, but response with the data included here in JSON:JSON:'''+function_response})
response = callLLM(messages,functions)
cr = classifyReponse(response)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U260'), dtype('float64')) -> None

In [301]:
cr

'content'

In [302]:
newContent = response['choices'][0]['message']['content']
newContent

'The slowest car in the dataset has a "acceleration" value of 24.8. It is a 1979 Peugeot 504 from origin 2.'

In [284]:
function_response

'{"mpg": 14.0, "cylinders": 8, "displacement": 340.0, "horsepower": "160", "weight": 3609, "acceleration": 8.0, "model year": 70, "origin": 1, "car name": "plymouth \'cuda 340"}'

json.dumps(df.sort_values('mpg',ascending=True).head(1).to_dict(orient='rows')[0])

In [33]:
def clearItOut():
    global messages
    messages = [
        {  
        "role": "system",
        "content": """You are a helpful assistant that advises a user on how to dress for running, based on the current weather.
          Only include information from the defined functions.
        Do not make up information, and keep your responses concise."""
    }
    ]

In [36]:


def classifyReponse(response):
    if response['choices'][0]['message']['content'] is not None:
        return 'content'
    elif 'function_call' in response['choices'][0]['message']:
        return 'function_call'
    
def prepResponse(response):
    cr = classifyReponse(response)
    print("classified as ",cr)
    if cr=='function_call':
        f = response['choices'][0]['message']['function_call']['name']
        kwargs = response['choices'][0]['message']['function_call']['arguments']
        function_response = globals()[f](kwargs)
        #return formatted message and logic to call llm again
        message = {
            "role": "function",
            "name": f,
            "content": ','.join(function_response),
        }
        action = 'llm'
        bot_message = None
    elif cr=='content':
        newContent = response['choices'][0]['message']['content']
        #return formatted message and logic to send to ui
        message = {
            "role": "assistant",
            "content": newContent,
        }
        action = 'ui'
        bot_message = newContent
    return bot_message, message,action

def callLLM(messages,functions):
    raw_message = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                functions = functions
            )
    return raw_message





In [13]:


with gr.Blocks(title="RunGearGuru",theme= "gstaff/whiteboard") as demo:
    gr.Markdown("Use the textbox below to submit your question to the Guru.")
    chatbot = gr.Chatbot()
    msg = gr.Textbox("What should i wear running in buffalo?") 
    clear = gr.Button("Clear")
    gr.Markdown("To learn about this demo, go to my Github at https://github.com/jonathanmanly/runGearGuru")

    def user(user_message, history):
        if ((dt.datetime.now()-startUpTime).seconds)/60>10:
            clearItOut()
        return "", history + [[user_message, None]]

    def bot(history):
        raw_message=' '
        messages.append({'role':'user',"content":history[-1][0]})
        raw_message = callLLM(messages,functions)
        print(raw_message)
        bot_message, history_message,action = prepResponse(raw_message)
        #print(2)
        messages.append(history_message)
        #print(3)
        if action =='llm':
            raw_message = callLLM(messages,functions)
            #print(4)
            bot_message, history_message,action = prepResponse(raw_message)
            #print(5)
            #print("at function call")
            #print(bot_message,history_message,action)
            messages.append(history_message)

            
        #determine here if it is function or content, then call function and produce content if so
        #bot_message = raw_message['choices'][0]['message']['content']
        #messages.append({"role":"assistant","content":bot_message})
        
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.001)
            yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    #clear.click(lambda: None, None, chatbot, queue=False)
    clear.click(clearItOut, None, chatbot, queue=False)
    
demo.queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
